In [1]:

import numpy as np
import pandas as pd

# file_path = "/Users/taegyu.hwang/Documents/tghwang_git_repo/ipitems_analysis/data/ml/view_list.csv"
file_path = "/Users/taegyu.hwang/Documents/tghwang_git_repo/ipitems_analysis/data/colley/view_list.csv"
user_id_to_idx = dict()
item_id_to_idx = dict()
user_idx_to_id = list()
item_idx_to_id = list()

view_user_ids = list()
view_item_ids = list()


view_df = pd.read_csv(file_path)
uidx, iidx = 0, 0
views_count = 0
users_count, items_count = 0, 0

for _, r in view_df.iterrows():
    user_id = int(r["user_id"])
    view_user_ids.append(user_id)
    item_id = int(r["item_id"])
    view_item_ids.append(item_id)
    views_count += 1
    if not user_id in user_id_to_idx:
        user_id_to_idx.update({user_id:uidx})
        user_idx_to_id.append(user_id)
        uidx += 1
    if not item_id in item_id_to_idx:
        item_id_to_idx.update({item_id:iidx})
        item_idx_to_id.append(item_id)
        iidx += 1
# end : for (views)

users_count = uidx
items_count = iidx

In [ ]:
views_matrix = np.full(shape=(users_count,items_count), fill_value=0.0, dtype=np.float32)
# decision_matrix = np.full(shape=(uidx,iidx), fill_value=np.nan, dtype=np.float32)
for idx in range(views_count):
    uidx = user_id_to_idx[view_user_ids[idx]]
    iidx = item_id_to_idx[view_item_ids[idx]]
    views_matrix[uidx][iidx] = 1.0
# end : for (views)

In [ ]:
from sklearn.decomposition import NMF

n_components = 80
## NNSVD++
model = NMF(n_components=n_components, init="random", solver="cd", beta_loss="frobenius",max_iter=250,)
view_W = model.fit_transform(X=views_matrix)
view_H = model.components_


In [ ]:
view_W

In [ ]:
view_H

In [ ]:
file_path = "/Users/taegyu.hwang/Documents/tghwang_git_repo/ipitems_analysis/data/ml/like_list.csv"

likes_user_ids_list = list()
likes_item_ids_list = list()
likes_count = 0

for _, r in pd.read_csv(file_path).iterrows():
    user_id = int(r["user_id"])
    item_id = int(r["item_id"])
    likes_user_ids_list.append(user_id)
    likes_item_ids_list.append(item_id)
    likes_count += 1
# end : for (likes)


In [ ]:
likes_matrix = np.zeros(shape=(users_count,items_count),dtype=np.float32,)

for idx in range(likes_count):
    user_id = likes_user_ids_list[idx]
    item_id = likes_item_ids_list[idx]
    if not user_id in user_id_to_idx:
        continue
    if not item_id in item_id_to_idx:
        continue
    uidx = user_id_to_idx[user_id]
    iidx = item_id_to_idx[item_id]
    likes_matrix[uidx,iidx] = 1
    # user_factor = W[uidx, :]
    # item_factor = H[:,iidx]
    
    #print(user_factor)
    # print(item_factor)
    
    #score = np.matmul(user_factor,item_factor)
    #print(score)
    # break
# end : for (likes)

v2l_W = model.fit_transform(
    X=views_matrix,
    y=likes_matrix,
    W=view_W,
    H=view_H,
    )
v2l_H = model.components_

print("1st ephoc")
for idx in range(likes_count):
    user_id = likes_user_ids_list[idx]
    item_id = likes_item_ids_list[idx]
    if not user_id in user_id_to_idx:
        continue
    if not item_id in item_id_to_idx:
        continue
    uidx = user_id_to_idx[user_id]
    iidx = item_id_to_idx[item_id]
    likes_matrix[uidx,iidx] = 1
    user_factor = view_W[uidx, :]
    item_factor = view_H[:,iidx]
    score = np.matmul(user_factor,item_factor)
    print(f"V: {score}")
    # print(user_factor)
    # print(item_factor)
    
    user_factor = v2l_W[uidx, :]
    item_factor = v2l_H[:,iidx]
    score = np.matmul(user_factor,item_factor)
    print(f"V to L: {score}")
    break
# end : for (likes)

In [ ]:
v2l_W = model.fit_transform(
    X=views_matrix,
    y=likes_matrix,
    W=v2l_W,
    H=v2l_H,
    )
v2l_H = model.components_

In [ ]:
# import sklearn.utils.optimize
# estimators = sklearn.utils.all_estimators(type_filter="regressor")

print("2nd ephoc")
for idx in range(likes_count):
    user_id = likes_user_ids_list[idx]
    item_id = likes_item_ids_list[idx]
    if not user_id in user_id_to_idx:
        continue
    if not item_id in item_id_to_idx:
        continue
    uidx = user_id_to_idx[user_id]
    iidx = item_id_to_idx[item_id]
    likes_matrix[uidx,iidx] = 1
    user_factor = view_W[uidx, :]
    item_factor = view_H[:,iidx]
    score = np.matmul(user_factor,item_factor)
    print(f"V: {score}")
    # print(user_factor)
    # print(item_factor)
    
    user_factor = v2l_W[uidx, :]
    item_factor = v2l_H[:,iidx]
    score = np.matmul(user_factor,item_factor)
    print(f"V to L: {score}")
    
    break
# end : for (likes)

In [ ]:
v2l_W = model.fit_transform(
    X=likes_matrix,
    W=v2l_W,
    H=v2l_H,
    )
v2l_H = model.components_

# import sklearn.utils.optimize
# estimators = sklearn.utils.all_estimators(type_filter="regressor")

print("n times ephoc")
err, cnt = 0.0, 0

for idx in range(likes_count):
    user_id = likes_user_ids_list[idx]
    item_id = likes_item_ids_list[idx]
    if not user_id in user_id_to_idx:
        continue
    if not item_id in item_id_to_idx:
        continue
    uidx = user_id_to_idx[user_id]
    iidx = item_id_to_idx[item_id]
    likes_matrix[uidx,iidx] = 1
    user_factor = view_W[uidx, :]
    item_factor = view_H[:,iidx]
    score = np.matmul(user_factor,item_factor)
    #print(f"V: {score}")
    ## print(user_factor)
    ## print(item_factor)
    
    user_factor = v2l_W[uidx, :]
    item_factor = v2l_H[:,iidx]
    score = np.matmul(user_factor,item_factor)
    # print(f"V to L: {score}")
    err += 1 - score
    cnt += 1
# end : for (likes)
err = err/cnt
print(f"mean err: {err}")

TruncatedSVD ㄱㄱ ㄱㄱ

In [ ]:
import numpy as np
import pandas as pd

file_path = "/Users/taegyu.hwang/Documents/tghwang_git_repo/ipitems_analysis/data/ml/view_list.csv"
user_id_to_idx = dict()
item_id_to_idx = dict()
user_idx_to_id = list()
item_idx_to_id = list()
view_user_ids = list()
view_item_ids = list()

view_df = pd.read_csv(file_path)
uidx, iidx = 0, 0
views_count = 0
users_count, items_count = 0, 0

for _, r in view_df.iterrows():
    user_id = int(r["user_id"])
    view_user_ids.append(user_id)
    item_id = int(r["item_id"])
    view_item_ids.append(item_id)
    views_count += 1
    if not user_id in user_id_to_idx:
        user_id_to_idx.update({user_id: uidx})
        user_idx_to_id.append(user_id)
        uidx += 1
    if not item_id in item_id_to_idx:
        item_id_to_idx.update({item_id: iidx})
        item_idx_to_id.append(item_id)
        iidx += 1
# end : for (views)

users_count = uidx
items_count = iidx
views_matrix = np.full(
    shape=(users_count, items_count), fill_value=0.0, dtype=np.float32
)
# decision_matrix = np.full(shape=(uidx,iidx), fill_value=np.nan, dtype=np.float32)
for idx in range(views_count):
    uidx = user_id_to_idx[view_user_ids[idx]]
    iidx = item_id_to_idx[view_item_ids[idx]]
    views_matrix[uidx][iidx] = 1.0
# end : for (views)

file_path = "/Users/taegyu.hwang/Documents/tghwang_git_repo/ipitems_analysis/data/ml/like_list.csv"

likes_user_ids_list = list()
likes_item_ids_list = list()
likes_count = 0

for _, r in pd.read_csv(file_path).iterrows():
    user_id = int(r["user_id"])
    item_id = int(r["item_id"])
    likes_user_ids_list.append(user_id)
    likes_item_ids_list.append(item_id)
    likes_count += 1
# end : for (likes)

likes_matrix = np.zeros(
    shape=(users_count, items_count),
    dtype=np.float32,
)

for idx in range(likes_count):
    user_id = likes_user_ids_list[idx]
    item_id = likes_item_ids_list[idx]
    if not user_id in user_id_to_idx:
        continue
    if not item_id in item_id_to_idx:
        continue
    uidx = user_id_to_idx[user_id]
    iidx = item_id_to_idx[item_id]
    likes_matrix[uidx, iidx] = 1
# end : for (likes)


from sklearn.decomposition import TruncatedSVD

n_components = 80
## NNSVD++
model = TruncatedSVD(
    n_components=n_components,
    algorithm="randomized",
    n_iter=250,
)
model = model.fit(X=views_matrix)
view_U = model.fit_transform(X=views_matrix)
view_S = model.singular_values_
view_V = model.components_


In [ ]:
view_P = view_U[0,:] * view_S
# view_P
print(view_S)
score = np.matmul(view_P, view_V[:,0])
score